----
----
----
----


# Adım — Bounding Box Görselleştirme GUI’si (`gui_bounding_box.py`)

In [1]:
%run gui_bounding_box.py

Output()

----
----
----
---


Bu kodun amacı, hazırladığımız YOLO veri setinin **doğru üretildiğini** hızlı ve güvenilir şekilde kontrol etmektir.  
Model eğitimine geçmeden önce, rastgele seçilen görseller üzerinde **label dosyalarından (YOLO .txt)** okunan bounding box’ları çizerek şu sorulara net cevap verir:

- Etiketler gerçekten “head” bölgelerine mi oturuyor?
- Train/Val/Test split’lerinde dosyalar doğru yerde mi?
- Bazı görsellerde bbox sayısı anormal mi (çok az / çok fazla)?
- Çok küçük bbox’lar (uzak kafa) dataset içinde nasıl dağılıyor?

Bu kontrol adımı, eğitim başlamadan önce yapılan bir **sanity check**’tir ve hatalı label/klasör yapısı varsa erken yakalamayı sağlar.



### 1) Kodun çalışma mantığı 

Kod, seçtiğin split’e göre (`train`, `val`, `test`) şu işlemleri yapar:

1. `images/<split>/` klasöründen bir görsel seçer (slider/next/prev/random ile).
2. Seçilen görselin ID’sine karşılık gelen label dosyasını bulur:
   - `labels/<split>/<ID>.txt`
3. Label dosyasındaki YOLO format satırlarını okur:
   - `class_id x_center y_center w h` (normalized)
4. Bu normalized değerleri piksel koordinatlarına çevirir:
   - `(x1, y1, x2, y2)` formatına getirir.
5. Görsel üzerinde bbox’ları çizerek ekranda gösterir.
6. Aynı anda bbox’ların detaylarını tablo olarak verir:
   - normalized değerler
   - piksel koordinatları
   - bbox genişlik/yükseklik (px)
   - alan (px²)


### 2) Kod nasıl kullanılır? (GUI üzerinden)

GUI’de temel kullanım şu şekildedir:

- **Split seçimi (train/val/test):**  
  Hangi veri kümesini incelemek istediğini seçersin.

- **Index slider + Prev/Next:**  
  Split içindeki görseller arasında sırayla gezinirsin.

- **Random:**  
  Rastgele bir örneğe atlar; dataset’i hızlı taramak için idealdir.

- **ID ile Git (Go):**  
  Belirli bir görsel ID’sini yazıp doğrudan o görsele gidersin.  
  Örn: `PartB_00042`


### 3) Filtreler ne işe yarar? (Özellikle kalabalık sınıflarda kritik)

- **Min area (px²):**  
  Çok küçük bbox’ları gizler. Uzakta kalan mini kafalar bazen görselde seçilmeyecek kadar küçük olur; bu filtre görüntüyü okunur yapar.

- **Min w(px) / Min h(px):**  
  Bbox genişliği veya yüksekliği belirli bir pikselin altındaysa gösterme. Özellikle “gürültü gibi” görünen bbox’ları elemek için kullanılır.

- **Max boxes:**  
  Çok kalabalık sahnelerde binlerce bbox çizmek Jupyter’ı yavaşlatabilir. Bu ayar en fazla kaç bbox çizileceğini sınırlar.

- **Sort (Original / Area ↑ / Area ↓):**  
  Bbox’ları alanına göre sıralar. Örneğin `Area ↓` seçince en büyük kafalar önce gelir; gözle kontrol hızlanır.

- **Bbox index göster:**  
  Her bbox’un yanına label satır index’i yazar. Tabloyla görseli eşleştirmeyi kolaylaştırır.

- **LineWidth:**  
  Kutuların çizgi kalınlığını ayarlar (görsel okunurluğu için).


### 4) Çıktıdan nasıl yorum yapacağız?

Bu GUI ile şu kontrolleri pratikte yapmış olursun:

- Kutular doğru yerde mi? (kafaya oturuyor mu)
- “Aşırı taşan” veya “alakasız yerde” bbox var mı?
- Çok küçük bbox yoğunluğu fazla mı? (min area ile bak)
- Bazı görsellerde bbox sayısı anormal mi? (üstteki “Toplam bbox” değeri)

Eğer burada her şey mantıklı görünüyorsa, veri seti hem format hem içerik açısından **eğitime hazır** demektir.

### 5) Nasıl çalıştırılır?

Bu dosya bir “Jupyter GUI” olduğu için en stabil kullanım şekli:

- Notebook’ta bir hücrede:  
  `%run gui_bounding_box.py`

Alternatif olarak terminalden de çalıştırılabilir; ancak widget etkileşimi notebook kadar sorunsuz olmayabilir.

Bu adım tamamlandıktan sonra bir sonraki aşama:  
**YOLO eğitimi (fine-tuning)** ve ardından **inference ile kişi sayma (bbox count)** olacaktır.
